In [ ]:
# Load Libraries
import pandas as pd
import json

*Change filename after uploading the data*

▶ Codeline: df = pd.read_csv("/content/sample_data/ENTER_FILE_NAME.csv")

In [ ]:
# Load your Bright Data output file
df = pd.read_csv("/content/sample_data/brightdata.csv")

# Define your search terms (case-insensitive)
keywords = ["jew", "netanyahu", "#gaza", "Zionist", "Israel", "settlers"]  # 👈 Edit this list as needed

# Storage for filtered results
parsed_rows = []
text_id = 1

# Iterate through each row
for _, row in df.iterrows():
    try:
        username = row.get("id", "")
        posts_raw = row.get("posts", "")

        if not posts_raw or pd.isna(posts_raw):
            continue

        # Safely parse JSON from the 'posts' column
        try:
            posts = json.loads(posts_raw)
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e}")
            print(f"Raw content preview: {posts_raw[:300]}\n")
            continue

        for post in posts:
            description = post.get("description", "")
            tweet_id = post.get("post_id", "")
            date_posted = post.get("date_posted", "")

            if description and tweet_id:
                if any(kw.lower() in description.lower() for kw in keywords):
                    parsed_rows.append({
                        "text_id": text_id,
                        "Text": description,
                        "tweet_id": tweet_id,
                        "Username": username,
                        "date_posted": date_posted
                    })
                    text_id += 1

    except Exception as e:
        print(f"Unexpected error: {e}")

# Save the filtered output
df_out = pd.DataFrame(parsed_rows)
df_out.to_csv("/content/sample_data/parsed_tweets_filtered.csv", index=False, encoding="utf-8")

# Shows you how much Tweets your collection already contains
print(f"Parsing complete. {len(df_out)} relevant tweets saved.")


▶ **Convert Output from Script above into Compatible Format for the Annotation**

In [ ]:
import pandas as pd

# Step 1: Load the uploaded file
# Adjust the path if necessary — this assumes you're using the default Colab folders
parsed_df = pd.read_csv("/content/sample_data/parsed_tweets_filtered.csv")

# Step 2: Transform into the required format
transformed_df = parsed_df.rename(columns={"tweet_id": "TweetID"})[["TweetID", "Username"]]

# Step 3: Save and download the result
output_file = "/content/sample_data/parsed_tweets_for_annotation.csv"
transformed_df.to_csv(output_file, index=False)